In [ ]:
!pip install czifile

import os
import glob
import czifile
import numpy as np
import cv2
import os
import shutil
import random

Read .czi files, separates slices (frames) and channels (channels) and save them in .png using the following folder structure:

-Raw-images (folder)
 |
 |--Individual_FOV_Scene# (folder) [Subcaegory-00]
     |
     |--Frame1 (folder) [Subcategory-01] 
        |
        |-- Channel1 (folder) [Subcategory-02] 
        |    |  Individual_FOV_Scene#_Channel1_Image1.png
        |    |  Individual_FOV_Scene#_Channel1_Image2.png
        |    |  ...
        |
        |-- Channel2 (folder) [Subcategory-02] 
        |    |  Individual_FOV_Scene#_Channel2_Image1.png
        |    |  Individual_FOV_Scene#_Channel2_Image2.png
        |    |  ...
        |
        |-- Channel3 (folder) [Subcategory-02] 
        |    |  Individual_FOV_Scene#_Channel3_Image1.png
        |    |  Individual_FOV_Scene#_Channel3_Image2.png
        |    |  ...
        |-- Channel4 (folder) [Subcategory-02] 
        |    |  Individual_FOV_Scene#_Channel4_Image1.png
        |    |  Individual_FOV_Scene#_Channel4_Image2.png
        |    |  ...
   

In [ ]:
def separate_frames_channels_from_czi(input_folder, output_folder):
    # Get all .czi files in the input folder
    czi_files = glob.glob(os.path.join(input_folder, "*.czi"))

    # Check if the output folder exists, if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for czi_file in czi_files:
        # Read the .czi file
        czi_data = czifile.imread(czi_file)

        # Create a subfolder in the output folder for each .czi file
        file_name = os.path.splitext(os.path.basename(czi_file))[0]
        file_folder = os.path.join(output_folder, file_name)
        os.makedirs(file_folder, exist_ok=True)

        # Iterate over frames
        for frame_index, frame_data in enumerate(czi_data):
            # Create a folder for each frame
            frame_folder = os.path.join(file_folder, f"Frame_{frame_index + 1}")
            os.makedirs(frame_folder, exist_ok=True)

            # Iterate over channels in each frame
            for channel_index, channel_data in enumerate(frame_data):
                # Create a folder for each channel
                channel_folder = os.path.join(frame_folder, f"Channel_{channel_index + 1}")
                os.makedirs(channel_folder, exist_ok=True)

                # Normalize channel data to 0-255 range
                channel_data = normalize_channel(channel_data)

                # Save each channel as individual .png images
                for image_index, image_data in enumerate(channel_data):
                    image_path = os.path.join(channel_folder, f"{file_name}_Frame{frame_index + 1}_Channel{channel_index + 1}_Image{image_index + 1}.png")
                    cv2.imwrite(image_path, image_data)
                    print(f"Saved: {image_path}")

    print("Frame and channel separation complete.")

# needed to work with non-RGB files
def normalize_channel(channel_data):
    min_val = np.min(channel_data)
    max_val = np.max(channel_data)

    normalized_data = (channel_data - min_val) * (255.0 / (max_val - min_val))
    normalized_data = normalized_data.astype(np.uint8)

    return normalized_data


# PATHS
input_folder = "/Users/user/path1"  # Replace with your input folder path
output_folder = "/Users/user/path2"  # Replace with your output folder path

separate_frames_channels_from_czi(input_folder, output_folder)

Once Slice ("z") and Channel ("c") data is separated, the files need to be sorted into the folder structure required by pix2pix in the ZeroCostDL4Mic notebook. 

Folder structure: 

-Training dataset
        |
        |-- Training_source
        |    |  Individual_FOV_Scene#_Channel1_Image1.png
        |    |  Individual_FOV_Scene#_Channel1_Image2.png
        |    |  ...
        |
        |-- Quality control dataset
        |    |  Individual_FOV_Scene#_Channel1_Image1.png
        |    |  Individual_FOV_Scene#_Channel1_Image2.png
        |    |  ...
        |
        |-- Data to be predicted
        |-- Results 
       
Since both training and quality control data is required to train and test the model, the files are split into 90% Training and 10% Quality_Control.

In [ ]:
import os
import shutil
import random

def copy_files(root_folder, output_folder):
    # Get all subfolders in the root folder
    subfolders = [f for f in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, f))]

    for subfolder in subfolders:
        frame_folder = os.path.join(root_folder, subfolder, "Frame_1")
        channel_folders = ["Channel_1", "Channel_2", "Channel_3", "Channel_4"]

        # Create target folders for training and quality control
        for i, channel_folder in enumerate(channel_folders, start=1):
            quality_control_folder = os.path.join(output_folder, f"Training_dataset_{i}")
            training_folder = os.path.join(output_folder, f"Quality_control_dataset_{i}")
            os.makedirs(training_folder, exist_ok=True)
            os.makedirs(quality_control_folder, exist_ok=True)

            channel_path = os.path.join(frame_folder, channel_folder)

            # Get all .png files ending in "_Image3" from the channel subfolder. The number at the end of "_Image()" dictates the slice
            files = [f for f in os.listdir(channel_path) if f.endswith("_Image3.png")]
            files.sort()  # Sort the files list

            # Copy files to the training dataset folder
            for file in files:
                src = os.path.join(channel_path, file)
                dst = os.path.join(training_folder, file)
                shutil.copy2(src, dst)
                print(f"Copied file: {src} -> {dst}")

            # Count the number of files in the training folder
            training_files = os.listdir(training_folder)
            num_files = len(training_files)

            # Move 10% of files to the quality control folder
            num_files_to_move = int(num_files * 0.1)  # Calculate 10% of the files
            files_to_move = random.sample(training_files, num_files_to_move)

            for file in files_to_move:
                src = os.path.join(training_folder, file)
                dst = os.path.join(quality_control_folder, file)
                shutil.move(src, dst)
                print(f"Moved file: {src} -> {dst}")


# PATHS
input_folder = "/Users/user/path2" # Replace with your input folder path
output_folder = "/Users/user/path3" # Replace with your output folder path
copy_files(root_folder, output_folder)


References: 
Lucas von Chamier*, Romain F. Laine*, Johanna Jukkala, Christoph Spahn, Daniel Krentzel, Elias Nehme, Martina Lerche, Sara Hernández-pérez, Pieta Mattila, Eleni Karinou, Séamus Holden, Ahmet Can Solak, Alexander Krull, Tim-Oliver Buchholz, Martin L Jones, Loic Alain Royer, Christophe Leterrier, Yoav Shechtman, Florian Jug, Mike Heilemann, Guillaume Jacquemet, Ricardo Henriques. Democratising deep learning for microscopy with ZeroCostDL4Mic. Nature Communications, 2021. DOI: https://doi.org/10.1038/s41467-021-22518-0

Phillip Isola, Jun-Yan Zhu, Tinghui Zhou, Alexei A. Efros. Image-to-Image Translation with Conditional Adversarial Networks. arXiv, v3.2018. DOI: https://doi.org/10.48550/arXiv.1611.07004
